In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score




In [111]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
data= pd.read_csv('HR_no_duplicate.csv')
df = data.copy()

In [12]:
 df.head() 
# df.department.unique()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,No,Yes,No,sales,low
1,0.80,0.86,5,262,6,No,Yes,No,sales,medium
2,0.11,0.88,7,272,4,No,Yes,No,sales,medium
3,0.72,0.87,5,223,5,No,Yes,No,sales,low
4,0.37,0.52,2,159,3,No,Yes,No,sales,low


In [22]:
target = 'satisfaction_level'
encode = ['Work_accident','left','promotion_last_5years','salary','department']

In [23]:
for col in encode:
    dummy = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df,dummy], axis=1)
    del df[col]


In [24]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident_No,Work_accident_Yes,left_No,left_Yes,promotion_last_5years_No,...,department_IT,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical
0,0.38,0.53,2,157,3,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
3,0.72,0.87,5,223,5,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
4,0.37,0.52,2,159,3,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0


In [19]:
# encoder = LabelEncoder()

# apply on df
# df['salary'] = encoder.fit_transform(df['salary'])
# df['Work_accident'] = encoder.fit_transform(df['Work_accident'])
# df['left'] = encoder.fit_transform(df['left'])
# df['promotion_last_5years'] = encoder.fit_transform(df['promotion_last_5years'])
# df['department'] = encoder.fit_transform(df['department'])

In [116]:
X = df.drop(['satisfaction_level'],axis=1)
y = df['satisfaction_level']

In [117]:
X_train, X_test, y_train, y_test = train_test_split(
 X, y, test_size=0.20, random_state=42)

In [118]:
model = XGBClassifier(
                    objective='binary:logistic',
                    max_depth= 4,
                    alpha= 10,
                    learning_rate= 1.0,
                    n_estimators=100)


model.fit(X_train, y_train)

[01:34:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=1.0, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [119]:
y_pred = model.predict(X_test)


In [120]:
# score = accuracy_score(y_test, y_pred)
cutoff = 0.7                              # decide on a cutoff limit
y_pred_classes = np.zeros_like(y_pred)    # initialise a matrix full with zeros
y_pred_classes[y_pred > cutoff] = 1 


y_test_classes = np.zeros_like(y_pred)
y_test_classes[y_test > cutoff] = 1

In [121]:
accuracy_score(y_test_classes, y_pred_classes)

0.5606502709462275

In [122]:
#save ML model


import pickle
pickle.dump(model, open("model.pkl", "wb"))